---
title: Information Theory
math:
    '\abs': '\left\lvert #1 \right\rvert'
    '\orm': '\left\lvert #1 \right\rvert'
    '\Set': '\left\{ #1 \right\}'
    '\set': '\operatorname{set}'   
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\SFM': '\operatorname{SFM}'
    '\utag': '\stackrel{\text{(#1)}}{#2}'
    '\uref': '\text{(#1)}'
    '\minimal': '\operatorname{minimal}'
---

In [ ]:
from __init__ import install_dependencies

await install_dependencies()

As mentioned in previous lectures, the following two lists `coin_flips` and `dice_rolls` simulate the random coin flips and rollings of a dice:

In [ ]:
# Do NOT modify any variables defined here because some tests rely on them
import random
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib widget
%reload_ext divewidgets

random.seed(0)  # for reproducible results.
num_trials = 200
coin_flips = ["H" if random.random() <= 1 / 2 else "T" for i in range(num_trials)]
dice_rolls = [random.randint(1, 6) for i in range(num_trials)]
print("coin flips: ", *coin_flips)
print("dice rolls: ", *dice_rolls)

**What is the difference of the two random processes? Can we say one process has more information content than the other?**

In this lab, we will use dictionaries to store distributions and then compute their information content using information theory, which was introduced by [*Claude E. Shannon*](https://en.wikipedia.org/wiki/Claude_Shannon) in his [seminal work in 1948](https://doi.org/10.1002%2Fj.1538-7305.1948.tb01338.x). It has [numerous applications](https://www.khanacademy.org/computing/computer-science/informationtheory): 

- *Compression* (to keep files small).
- *Communications* (to send data to mobile phones).
- *Cybersecurity* (to protect data using provably secure methods).
- *Machine learning* (to identify relevant features for learning).

The first [conference on Information Theory](https://en.wikipedia.org/wiki/IEEE_International_Symposium_on_Information_Theory) was held in London in 1950, where [Alan Turing also contributed](https://www.turing.org.uk/sources/info50turing.html).

## Entropy

The following code shown in the lecture uses a dictionary to store the [(empirical) distribution](https://en.wikipedia.org/wiki/Empirical_distribution_function#:~:text=The%20empirical%20distribution%20function%20is,to%20the%20Glivenko%E2%80%93Cantelli%20theorem.) for a sequence efficiently without storing outcomes with zero counts:

In [ ]:
# Do NOT modify any variables defined here because some tests rely on them
def distribute(seq):
    """Returns a dictionary where each value in a key-value pair is
    the probability of the associated key occurring in the sequence.
    """
    p = {}
    for i in seq:
        p[i] = p.get(i, 0) + 1 / len(seq)
    return p


# tests
coin_flips_dist = distribute(coin_flips)
dice_rolls_dist = distribute(dice_rolls)
print("Distribution of coin flips:", coin_flips_dist)
print("Distribution of dice rolls:", dice_rolls_dist)

::::{prf:definition} probability mass function (pmf)

A distribution on discrete outcomes is a collection of non-negative probability masses

$$
\begin{align}
\M{p} &=\begin{bmatrix} p_{i} \end{bmatrix}_{i\in \mc{S}} && \text{such that}\\
\sum_{i\in \mc{S}} p_i &= 1, && \text{where}
\end{align}
$$

- $\mc{S}$ is the set of distinct outcomes.
- $p_i$ denotes the probability (chance) of seeing outcome $i$.

::::

You can verify that the probabilities $p_i$'s have to sum to $1$ for the empirical distributions of coin flips and dice rolls:

In [ ]:
assert math.isclose(sum(coin_flips_dist.values()), 1) and math.isclose(
    sum(dice_rolls_dist.values()), 1
)

**How to measure the information content?**

::::{card}
:header: Entropy
:footer: [(open in new tab)](https://www.youtube.com/embed/2s3aJfRr9gE)

:::{iframe} https://www.youtube.com/embed/2s3aJfRr9gE
:align: center
:width: 100%

:::

::::{prf:definition} Entropy
:label: def:entropy

In information theory, the information content of a distribution is measured by its [*entropy*](https://en.wikipedia.org/wiki/Entropy_(information_theory)) defined as:

$$ 
\begin{aligned} 
H(\M{p}) &:= \sum_{i\in \mc{S}} p_i \overbrace{\log_2 \tfrac{1}{p_i}}^{\text{called surprise} } \\ 
&= - \sum_{i\in \mc{S}} p_i \log_2 p_i \kern1em \text{(bits)} \end{aligned}  
$$

with $p_i \log_2 \frac{1}{p_i} = 0$ if $p_i = 0$ because $\lim_{x\downarrow 0} x \log_2 \frac1x = 0$.

::::

For instance, if $\M{p}=(p_{H},p_{T})=(0.5,0.5)$, then

$$
\begin{aligned} H(\M{p}) &= 0.5 \log_2 \frac{1}{0.5} + 0.5 \log_2 \frac{1}{0.5} \\ &= 0.5 + 0.5 = 1  \text{ bit,}\end{aligned} 
$$

i.e., an outcome of a fair coin flip has one bit of information content, as expected.

On the other hand, if $\M{p}=(p_{H},p_{T})=(1,0)$, then

$$
\begin{aligned} H(\M{p}) &= 1 \log_2 \frac{1}{1} + 0 \log_2 \frac{1}{0} \\ &= 0 + 0 = 0  \text{ bits,}\end{aligned} 
$$

i.e., an outcome of a biased coin flip that always comes up head has no information content, again as expected.

::::{exercise}
:label: ex:entropy

Define a function `entropy` that
- takes a distribution $\M{p}$ as its argument, and
- returns the entropy $H(\M{p})$.

Handle the case when $p_i=0$, e.g., using the short-circuit evaluation of logical `and`.

::::

In [ ]:
def entropy(dist):
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
assert math.isclose(entropy({"H": 0.5, "T": 0.5}), 1)
assert math.isclose(entropy({"H": 1, "T": 0}), 0)
assert math.isclose(entropy(dict.fromkeys(range(1, 7), 1 / 6)), math.log2(6))

In [ ]:
# HIDDEN

## Uniform distribution maximizes entropy

Intuitively,
- for large enough numbers of fair coin flips, we should have $\mc{S}=\{H,T\}$ and $p_H=p_T=0.5$, i.e., equal chance for head and tail.
- for large enough numbers of fair dice rolls, we should have $p_i=\frac16$ for all $i\in \mc{S}=\{1,2,3,4,5,6\}$.

In [ ]:
def plot_distribution(seq, *args, **kwargs):
    fig = plt.figure(*args, **kwargs)
    plt.xlabel("Outcomes")
    plt.title("Distribution")
    plt.ylim(0, 1)
    dist_plot = ()

    @widgets.interact(
        n=widgets.IntSlider(
            value=1,
            min=1,
            max=len(seq),
            step=1,
            description="n:",
            continuous_update=False,
        )
    )
    def plot(n):
        plt.figure(fig.number)
        nonlocal dist_plot
        dist_plot and dist_plot.remove()
        dist = distribute(seq[:n])
        dist_plot = plt.stem(dist.keys(), dist.values())
        plt.show()

In [ ]:
plot_distribution(coin_flips, num=1, clear=True)

In [ ]:
plot_distribution(dice_rolls, num=2, clear=True)

::::{prf:definition} Uniform
:label: def:uniform-distribution

A distribution is called a *uniform distribution* if all its distinct outcomes have the same probability of occurring, i.e.,

$$ p_i = \frac{1}{|\mc{S}|}\kern1em  \text{for all }i\in \mc{S},  $$

where $|\mc{S}|$ is the mathematical notation to denote the size of the set $\mc{S}$.

::::

::::{exercise}
:label: ex:uniform

Define a function `uniform` that
- takes a sequence of possibly duplicate outcomes, and
- returns a uniform distribution of the distinct outcomes.

::::

In [ ]:
def uniform(outcomes):
    """Returns the uniform distribution (dict) over distinct items in outcomes."""
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
assert uniform("HT") == {"H": 0.5, "T": 0.5}
assert uniform("HTH") == {"H": 0.5, "T": 0.5}
fair_dice_dist = uniform(range(1, 7))
assert all(
    math.isclose(fair_dice_dist[k], v)
    for k, v in {
        1: 0.16666666666666666,
        2: 0.16666666666666666,
        3: 0.16666666666666666,
        4: 0.16666666666666666,
        5: 0.16666666666666666,
        6: 0.16666666666666666,
    }.items())

In [ ]:
# HIDDEN

**What is the entropy for uniform distributions?**

By definition,

$$ \begin{aligned} H(\M{p}) &:= \sum_{i\in \mc{S}} p_i \log_2 \tfrac{1}{p_i} \\ &= \sum_{i\in \mc{S}} \frac{1}{|\mc{S}|} \log_2 |\mc{S}| = \log_2 |\mc{S}|  \kern1em \text{(bits)} \end{aligned}  $$

Entropy reduces to the formula in [Lab 1](../Lab1/Binary_Code.ipynb) regarding the number of bits required to encode a set of integers or characters. It is the maximum possible entropy for a given finite set of possible outcomes.

Use this result to test whether you have implemented both `entropy` and `uniform` correctly:

In [ ]:
assert all(
    math.isclose(entropy(uniform(range(n))), math.log2(n)) for n in range(1, 100)
)

## Joint distribution and its entropy

If we duplicate a sequence of outcomes, the total information content should remain unchanged, *not* doubled, because the duplicate contains the same information as the original. We will verify this fact by creating a [joint distribution](https://en.wikipedia.org/wiki/Joint_probability_distribution).

::::{prf:definition} joint pmf

The joint distribution on discrete outcomes is a collection of non-negative probability masses

$$
\begin{align}
\M{p} &=\begin{bmatrix} p_{ij} \end{bmatrix}_{i\in \mc{S},j\in \mc{T}}, && \text{such that}\\
\sum_{i\in \mc{S}} \sum_{j\in \mc{T}} p_{ij} &= 1, && \text{where}
\end{align}
$$

- $\mc{S}$ and $\mc{T}$ are sets of outcomes.
- $p_{ij}$ is the chance we see outcomes $i$ and $j$ simultaneously. The subscript $ij$ in $p_{ij}$ denotes a tuple $(i,j)$, NOT the multiplication $i\times j$.

::::

::::{exercise}
:label: ex:jointly-distribute

Define a function `jointly_distribute` that 
- takes two sequences, `seq1` and `seq2`, of outcomes with the same length, and
- returns the joint distribution represented as a dictionary where each key-value pair has the key being a tuple `(i,j)` associated with the probability $p_{ij}$ of seeing `(i,j)` in `zip(seq1,seq2)`.

::::

In [ ]:
def jointly_distribute(seq1, seq2):
    """Returns the joint distribution of the tuple (i,j) of outcomes from zip(seq1,seq2)."""
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
assert jointly_distribute("HT", "HT") == {("H", "H"): 0.5, ("T", "T"): 0.5}
assert jointly_distribute("HHTT", "HTHT") == {
    ("H", "H"): 0.25,
    ("H", "T"): 0.25,
    ("T", "H"): 0.25,
    ("T", "T"): 0.25}
coin_flips_duplicate_dist = {
    ("T", "T"): 0.5350000000000004,
    ("H", "H"): 0.4650000000000003}
coin_flips_duplicate_ans = jointly_distribute(coin_flips, coin_flips)
assert all(
    math.isclose(coin_flips_duplicate_ans[i], pi)
    for i, pi in coin_flips_duplicate_dist.items())

In [ ]:
# HIDDEN

If you have implemented `entropy` and `jointly_distribute` correctly, you can verify that duplicating a sequence will give the same entropy.

In [ ]:
assert math.isclose(
    entropy(jointly_distribute(coin_flips, coin_flips)), entropy(distribute(coin_flips))
)
assert math.isclose(
    entropy(jointly_distribute(dice_rolls, dice_rolls)), entropy(distribute(dice_rolls))
)

However, for two sequences generated independently, the joint entropy is roughly the sum of the individual entropies.

In [ ]:
coin_flips_entropy = entropy(distribute(coin_flips))
dice_rolls_entropy = entropy(distribute(dice_rolls))
cf_dr_entropy = entropy(jointly_distribute(coin_flips, dice_rolls))
print(
    f"""Entropy of coin flip: {coin_flips_entropy}
Entropy of dice roll: {dice_rolls_entropy}
Sum of the above entropies: {coin_flips_entropy + dice_rolls_entropy}
Joint entropy: {cf_dr_entropy}"""
)

## Conditional distribution and entropy

::::{prf:definition} conditional pmf

A [conditional distribution](https://en.wikipedia.org/wiki/Conditional_probability_distribution) on discrete outcomes is a collection of non-negative probability masses

$$
\begin{align}
\M{q} &:=\begin{bmatrix} q_{j|i} \end{bmatrix}_{i\in \mc{S}, j\in \mc{T}} && \text{such that}\\
\sum_{j\in \mc{T}} q_{j|i} &= 1 \kern1em \text{for all }i\in \mc{S}, && \text{where}
\end{align}
$$ 

where 
- $\mc{S}$ and $\mc{T}$ are two sets of distinct outcomes, and
- $q_{j|i}$ denotes the probability (chance) of seeing outcome $j$ given the condition that outcome $i$ is observed.

::::

For example, suppose we want to compute the distribution of coin flips given dice rolls, then the following assign `q_H_1` and `q_T_1` to the values $q_{H|1}$ and $q_{T|1}$ respectively:

In [ ]:
coin_flips_1 = [j for i, j in zip(dice_rolls, coin_flips) if i == 1]
q_H_1 = coin_flips_1.count("H") / len(coin_flips_1)
q_T_1 = coin_flips_1.count("T") / len(coin_flips_1)
print("Coin flips given dice roll is 1:", coin_flips_1)
print(
    'Distribution of coin flip given dice roll is 1: {{ "H": {}, "T": {}}}'.format(
        q_H_1, q_T_1
    )
)
assert math.isclose(q_H_1 + q_T_1, 1)

Note that `q_H_1 + q_T_1` is 1 as expected. Similarly, we can assign `q_H_2` and `q_T_2` to the values $q_{H|2}$ and $q_{T|2}$ respectively.

In [ ]:
coin_flips_2 = [j for i, j in zip(dice_rolls, coin_flips) if i == 2]
q_H_2 = coin_flips_2.count("H") / len(coin_flips_2)
q_T_2 = coin_flips_2.count("T") / len(coin_flips_2)
print("Coin flips given dice roll is 2:", coin_flips_2)
print(
    'Distribution of coin flip given dice roll is 2: {{ "H": {}, "T": {}}}'.format(
        q_H_2, q_T_2
    )
)
assert math.isclose(q_H_2 + q_T_2, 1)

Finally, we want to store the conditional distribution as a nested dictionary so that `q[i]` points to the distribution 

$$\begin{bmatrix} q_{j|i} \end{bmatrix}_{j\in \mc{T}} \kern1em \text{for }i\in \mc{S}.$$

In [ ]:
q = {}
q[1] = dict(zip("HT", (q_H_1, q_T_1)))
q[2] = dict(zip("HT", (q_H_2, q_T_2)))
...  # entries for q[i] other possible outcomes i of the dice rolls.
q

::::{exercise}
:label: ex:conditionally-distribute

Define a function `conditionally_distribute` that
- takes two sequences `seq1` and `seq2` of outcomes of the same length, and
- returns the conditional distribution of `seq2` given `seq1` in the form of a nested dictionary efficiently without storing the unobserved outcomes.

In the above example, `seq1` is `dice_rolls` while `seq2` is `coin_flips`.

:::{hint}
:class: dropdown

For an efficient implementation without traversing the input sequences too many times, consider using the following solution template and the `setdefault` method.
```python
def conditionally_distribute(seq1, seq2):
    q, count = {}, {}  # NOT q = count = {}
    for i in seq1:
        count[i] = count.get(i, 0) + 1
    for i, j in zip(seq1, seq2):
        q[i][j] = ...
    return q
```
:::

::::

In [ ]:
def conditionally_distribute(seq1, seq2):
    """Returns the conditional distribution q of seq2 given seq1 such that
    q[i] is a dictionary for observed outcome i in seq1 and
    q[i][j] is the probability of observing j in seq2 given the
    corresponding outcome in seq1 is i."""
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
cf_given_dr_dist = {
    4: {"T": 0.5588235294117647, "H": 0.4411764705882353},
    1: {"T": 0.46511627906976744, "H": 0.5348837209302325},
    3: {"H": 0.5135135135135135, "T": 0.4864864864864865},
    6: {"H": 0.5454545454545454, "T": 0.45454545454545453},
    2: {"T": 0.7586206896551724, "H": 0.2413793103448276},
    5: {"T": 0.5416666666666666, "H": 0.4583333333333333}}
cf_given_dr_ans = conditionally_distribute(dice_rolls, coin_flips)
assert all(
    math.isclose(cf_given_dr_ans[i][j], v)
    for i, d in cf_given_dr_dist.items()
    for j, v in d.items())

In [ ]:
# HIDDEN

::::{prf:definition} Conditional entropy
:label: def:conditional-entropy

The [*conditional entropy*](https://en.wikipedia.org/wiki/Conditional_entropy) is defined for a conditional distribution $\M{q}=\begin{bmatrix} q_{j|i} \end{bmatrix}_{i\in \mc{S},j\in\mc{T}}$ and a distribution $\M{p}=\begin{bmatrix} p_i \end{bmatrix}_{i\in \mc{S}}$ as follows:

$$ H(\M{q}|\M{p}) = \sum_{i\in \mc{S}} p_i \sum_{j\in \mc{T}} q_{j|i} \log_2 \frac{1}{q_{j|i}}, $$
where, by convention:  
1. The summand of the outer sum is 0 if $p_i=0$ (regardless of the values of $q_{j|i}$).
2. The summand of the inner sum is 0 if $q_{j|i}=0$.

::::

::::{exercise}
:label: ex:conditional-entropy

Define a function `conditional_entropy` that
- takes 
  1. a distribution `p` as its first argument,
  2. a conditional distribution `q` as its second argument, and
- returns the conditional entropy $H(\M{q}|\M{p})$.

Handle the cases when $p_i=0$ and $q_{j|i}=0$ for some indices $i$ and $j$.

::::

In [ ]:
# YOUR CODE HERE
raise NotImplementedError

In [ ]:
# tests
cf_given_dr_dist = {
    4: {"T": 0.5588235294117647, "H": 0.4411764705882353},
    1: {"T": 0.46511627906976744, "H": 0.5348837209302325},
    3: {"H": 0.5135135135135135, "T": 0.4864864864864865},
    6: {"H": 0.5454545454545454, "T": 0.45454545454545453},
    2: {"T": 0.7586206896551724, "H": 0.2413793103448276},
    5: {"T": 0.5416666666666666, "H": 0.4583333333333333}}
assert (
    conditional_entropy(
        {"H": 0.5, "T": 0.5}, {"H": {"H": 0.5, "T": 0.5}, "T": {"H": 0.5, "T": 0.5}})
    == 1)
assert (
    conditional_entropy(
        {"H": 0, "T": 1}, {"H": {"H": 0.5, "T": 0.5}, "T": {"H": 0.5, "T": 0.5}})
    == 1)
assert (
    conditional_entropy(
        {"H": 0.5, "T": 0.5}, {"H": {"H": 1, "T": 0}, "T": {"H": 0, "T": 1}})
    == 0)
assert (
    conditional_entropy(
        {"H": 0.5, "T": 0.5}, {"H": {"H": 1, "T": 0}, "T": {"H": 0.5, "T": 0.5}})
    == 0.5)
assert math.isclose(
    conditional_entropy(dice_rolls_dist, cf_given_dr_dist), 0.9664712793722372)

In [ ]:
# HIDDEN

The joint probability $p_{ij}$ over $i\in \mc{S}$ and $j\in \mc{T}$ can be calculated as follows

$$p_{ij} = p_{i} q_{j|i}$$

where, as introduced earlier, $p_i$ is the probability of $i$ and $q_{j|i}$ is the conditional probability of $j$ given $i$.

::::{exercise}
:label: ex:joint-distribution

Define a function `joint_distribution` that
- takes the distribution $p$ and conditional distribution $q$ as arguments, and
- returns their joint distribution.

::::

In [ ]:
def joint_distribution(p, q):
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
assert joint_distribution(
    {"H": 0.5, "T": 0.5}, {"H": {"H": 0.5, "T": 0.5}, "T": {"H": 0.5, "T": 0.5}}) == {("H", "H"): 0.25, ("H", "T"): 0.25, ("T", "H"): 0.25, ("T", "T"): 0.25}
assert joint_distribution(
    {"H": 0, "T": 1}, {"H": {"H": 0.5, "T": 0.5}, "T": {"H": 0.5, "T": 0.5}}) == {("H", "H"): 0.0, ("H", "T"): 0.0, ("T", "H"): 0.5, ("T", "T"): 0.5}
assert joint_distribution(
    {"H": 0.5, "T": 0.5}, {"H": {"H": 1, "T": 0}, "T": {"H": 0, "T": 1}}) == {("H", "H"): 0.5, ("H", "T"): 0.0, ("T", "H"): 0.0, ("T", "T"): 0.5}, {
    "H": 0.5,
    "T": 0.5}

In [ ]:
# HIDDEN

Finally, a fundamental information identity relating the joint and conditional entropies is the [*chain rule*](https://en.wikipedia.org/wiki/Conditional_entropy#Chain_rule):

::::{prf:theorem}
:label: thm:chain-rule

The joint entropy is equal to

$$ H(\M{p}) + H(\M{q}|\M{p})$$

for any distribution $\M{p}$ over outcome $i\in \mc{S}$ and conditional distribution $\M{q}$ over outcome $j\in \mc{T}$ given outcome $i \in \mc{S}$. 

::::

If you have implemented `jointly_distribute`, `conditionally_distribute`, `entropy`, and `conditional_entropy` correctly, we can verify the identity as follows.

In [ ]:
def validate_chain_rule(seq1, seq2):
    p = distribute(seq1)
    q = conditionally_distribute(seq1, seq2)
    pq = jointly_distribute(seq1, seq2)
    H_pq = entropy(pq)
    H_p = entropy(p)
    H_q_p = conditional_entropy(p, q)
    print(
        f"""Entropy of seq1: {H_p}
Conditional entropy of seq2 given seq1: {H_q_p}
Sum of the above entropies: {H_p + H_q_p}
Joint entropy: {H_pq}"""
    )
    assert math.isclose(H_pq, H_p + H_q_p)

In [ ]:
validate_chain_rule(coin_flips, coin_flips)

In [ ]:
validate_chain_rule(dice_rolls, coin_flips)